## Part 3: Comparing Manual K2 vs bnstruct

In this section we **compare** the two learned structures on the same datasets (Asia, Child, Ruiz) using:

1. **Structural metrics**  
   - **Structural Hamming Distance** (SHD)  
   - **Edge counts**  
   - **Precision / Recall / F₁** (treating one network as “reference”)

2. **Statistical scores**  
   - **Log‐likelihood**  
   - **BDeu** (marginal likelihood under BD equivalent uniform prior)  
   - **BIC** and **AIC**  

We use the **bnlearn** package to compute all of these.

---

### Libraries & Data

In [ ]:
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
# Required packages
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
if(!requireNamespace("bnlearn", quietly=TRUE))    install.packages("bnlearn")
if(!requireNamespace("bnstruct", quietly=TRUE))   install.packages("bnstruct")
if(!requireNamespace("Rgraphviz", quietly=TRUE))  BiocManager::install("Rgraphviz")
if(!requireNamespace("igraph", quietly=TRUE))     install.packages("igraph")

library(bnlearn)
library(bnstruct)
library(Rgraphviz)
library(igraph)

#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
# Load Part 1 networks (manual K2) and raw data
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

# (Assume you saved the results of Part 1 as bn1_asia, bn1_child, bn1_ruiz 
#  each of class ‘bn’ from bnlearn.)

# Load the original data for scoring
asia_df  <- asia()
child_df <- child_NA_5000()
# Ruiz dummy:
ruiz_df_raw <- data.frame(
  x1=c(1,1,0,1,0,0,1,0,1,0),
  x2=c(0,1,0,1,0,1,1,0,1,0),
  x3=c(0,1,1,1,0,1,1,0,1,0)
)
ruiz_df <- ruiz_df_raw + 1  # shift 0/1 → 1/2

Warning message in install.packages("bnlearn"):
“l'installazione del pacchetto ‘bnlearn’ ha uno stato di uscita non-zero”
Aggiornamento indice HTML dei pacchetti in '.Library'

Making 'packages.html' ...
 fatto



ERROR: Error in library(bnlearn): non c'è alcun pacchetto chiamato ‘bnlearn’


#### Convert bnstruct DAGs → bnlearn objects

In [ ]:
# helper to convert a bnstruct network to bnlearn::bn
bnstruct_to_bnlearn <- function(bnstruct_net) {
  vars <- variables(bnstruct_net)
  adj  <- dag(bnstruct_net)
  bn   <- empty.graph(nodes=vars)
  for(i in seq_along(vars)) for(j in seq_along(vars)) {
    if(adj[i,j]==1) bn <- set.arc(bn, from=vars[i], to=vars[j])
  }
  bn
}

# Load Part 2 networks (from your notebook)
asia_bs  <- learn.network( asia(), algo="hc",
                           scoring.func="BDeu",
                           layering=1:num.variables(asia()),
                           max.fanin=2 )
child_bs <- { d<-child(); d<-impute(d);
              learn.network(d, algo="hc", scoring.func="BDeu",
                            layering=1:num.variables(d),
                            max.fanin=2, use.imputed.data=TRUE) }
ruiz_bs  <- { d<-BNDataset(data=ruiz_df, discreteness=rep("d",3),
                           variables=c("x1","x2","x3"),
                           node.sizes=rep(2,3));
              learn.network(d, algo="hc", scoring.func="BDeu",
                            layering=1:3, max.fanin=2) }

bn2_asia  <- bnstruct_to_bnlearn(asia_bs)
bn2_child <- bnstruct_to_bnlearn(child_bs)
bn2_ruiz  <- bnstruct_to_bnlearn(ruiz_bs)

#### Structural Hamming Distance (SHD)
The Structural Hamming Distance between two DAGs $(G_1,G_2)$ is
$$
\mathrm{SHD}(G_1,G_2)=\bigl|{(i,j):A_1(i,j)\neq A_2(i,j)}\bigr|.
$$

In [ ]:
shd_asia  <- shd(bn1_asia,  bn2_asia)
shd_child <- shd(bn1_child, bn2_child)
shd_ruiz  <- shd(bn1_ruiz,  bn2_ruiz)

data.frame(
  Dataset = c("Asia","Child","Ruiz"),
  SHD     = c(shd_asia, shd_child, shd_ruiz)
)

#### Edge Counts & Precision/Recall

Let
$$
E_1,E_2
$$
be the arc‐sets.  Then:
$$
\text{TP} = |E_1\cap E_2|,\quad
\text{FP} = |E_2\setminus E_1|,\quad
\text{FN} = |E_1\setminus E_2|.
$$
+ Precision ($= \tfrac{TP}{TP+FP}$,)
+ Recall ($= \tfrac{TP}{TP+FN}$,)
+ F1 Score ($=2\frac{PR}{P+R}$).

In [ ]:
compare_stats <- function(bn_ref, bn_learn) {
  cm <- compare(bn_ref, bn_learn, report="both") 
  with(cm, data.frame(TP=tp, FP=fp, FN=fn,
                      Precision=tp/(tp+fp),
                      Recall=tp/(tp+fn),
                      F1=2*tp/(2*tp+fp+fn)))
}

pr_asia  <- compare_stats(bn1_asia,  bn2_asia)
pr_child <- compare_stats(bn1_child, bn2_child)
pr_ruiz  <- compare_stats(bn1_ruiz,  bn2_ruiz)

rbind(
  cbind(Dataset="Asia",  pr_asia),
  cbind(Dataset="Child", pr_child),
  cbind(Dataset="Ruiz",  pr_ruiz)
)

#### Statistical Scores

For a BN (G) and data (D) of size (N):
+ Log‐likelihood
($\displaystyle \ell(G\mid D)=\sum_{d\in D}\log P(d\mid\hat\Theta_G)$)
+ BDeu
($\log P(D\mid G)$) under a Dirichlet prior (($\mathrm{iss}=1$))
+ BIC
($\displaystyle \mathrm{BIC}(G)
= 2\ell(G\mid D)-|\Theta_G|\log N$)
+ AIC
($\displaystyle \mathrm{AIC}(G)
= 2\ell(G\mid D)-2|\Theta_G|$)

In [ ]:
score_table <- function(bn, data) {
  N <- nrow(data)
  list(
    loglik = score(bn, data, type="loglik"),
    BDeu   = score(bn, data, type="bde",   iss=1),
    BIC    = score(bn, data, type="bic"),
    AIC    = score(bn, data, type="aic")
  )
}

scores <- rbind(
  cbind(Dataset="Asia",  do.call(cbind, score_table(bn2_asia,  asia_df))),
  cbind(Dataset="Child", do.call(cbind, score_table(bn2_child, child_df))),
  cbind(Dataset="Ruiz",  do.call(cbind, score_table(bn2_ruiz,  ruiz_df)))
)
scores